# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url	slides_url
2024-05-15	Talk2Care: An LLM-based Voice Assistant for Communication between Healthcare Providers and Older Adults	Proceedings of the ACM on Interactive, Mobile, Wearable and Ubiquitous Technologies		Yang, Z., Xu, X., Yao, B., Rogers, E., Zhang, S., Intille, S., ... & Wang, D. (2024). Talk2care: An llm-based voice assistant for communication between healthcare providers and older adults. Proceedings of the ACM on Interactive, Mobile, Wearable and Ubiquitous Technologies, 8(2), 1-35.	talk2care	https://doi.org/10.1145/3659625	
2023-02-16	Companion: A Pilot Randomized Clinical Trial to Test an Integrated Two-Way Communication and Near-Real-Time Sensing System for Detecting and Modifying Daily Inactivity among Adults >60 Years—Design and Protocol	MDPI		Arguello, D., Rogers, E., Denmark, G. H., Lena, J., Goodro, T., Anderson-Song, Q., Cloutier, G., Hillman, C. H., Kramer, A. F., Castaneda-Sceppa, C., & John, D. (2023). Comp

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [21]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url,slides_url
0,2024-05-15,Talk2Care: An LLM-based Voice Assistant for Co...,"Proceedings of the ACM on Interactive, Mobile,...",NaN,"Ziqi Yang, Xuhai Xu, Bingsheng Yao, <strong>Et...",talk2care,https://doi.org/10.1145/3659625,NaN
1,2023-02-16,Companion: A Pilot Randomized Clinical Trial t...,MDPI,NaN,"Arguello, D., <strong>Rogers, E.</strong>, Den...",companion,https://doi.org/10.3390/s23042221,NaN
2,2021-05-04,Influence of Opioid Prescription Policy on Ove...,Journal of General Internal Medicine,NaN,"Harder, V. S., Plante, T. B., Koh, I., <strong...",opioids,https://doi.org/10.1007/s11606-021-06831-4,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [22]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [24]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.slides_url)) > 5:
        md += "\nslidesurl: '" + item.slides_url + "'"

    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"

    if len(str(item.slides_url)) > 5:
        md += "\n[Download slides here](" + item.slides_url + ")\n" 

    #if len(str(item.paper_url)) > 5:
    #    md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    #md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [22]:
!ls ../_publications/

2021-05-04-opioids.md   2023-02-16-companion.md 2024-05-15-talk2care.md


In [18]:
!cat ../_publications/*.md

---
title: "Influence of Opioid Prescription Policy on Overdoses and Related Adverse Effects in a Primary Care Population"
collection: publications
permalink: /publication/2021-05-04-opioids
date: 2021-05-04
venue: 'Journal of General Internal Medicine'
paperurl: 'https://doi.org/10.1007/s11606-021-06831-4'
citation: 'Harder, V. S., Plante, T. B., Koh, I., Rogers, E. B., Varni, S. E., Villanti, A. C., Brooklyn, J. R., &amp; Fairfield, K. M. (2021). Influence of Opioid Prescription Policy on Overdoses and Related Adverse Effects in a Primary Care Population. Journal of general internal medicine, 36(7), 2013–2020.'
---
[Download paper here](https://doi.org/10.1007/s11606-021-06831-4)

Recommended citation: Harder, V. S., Plante, T. B., Koh, I., Rogers, E. B., Varni, S. E., Villanti, A. C., Brooklyn, J. R., & Fairfield, K. M. (2021). Influence of Opioid Prescription Policy on Overdoses and Related Adverse Effects in a Primary Care Population. Journal of general internal medicine, 36(7), 2